### Libraries and definitions

In [3]:
%matplotlib inline
import shutil, os, sys
from prody import *
from pylab import *
from matplotlib import pyplot as plt
import numpy as np
import mrcfile
from scipy.ndimage import gaussian_filter

In [4]:
chimera_bin='/Applications/Chimera.app/Contents/MacOS/chimera'

# LocalResDomains
Here we segment the local resolution map into domains.

## using Segger
See following [link](http://plato.cgl.ucsf.edu/pipermail/chimera-users/2014-January/009553.html) for some info about how to proceed. The Segger python tools can be found here: `/Applications/Chimera.app/Contents/Resources/share/Segger`.

It is hosted at [SLAC](https://cryoem.slac.stanford.edu/ncmi/resources/software/segger)

In [8]:
input_dir = '/Users/fpoitevi/gdrive/cryoEM/Projects/20181005-rib-TEM4/processing/LocalRes/bin2_of_bin6mb_B'
body_dir  = '/Users/fpoitevi/gdrive/cryoEM/Projects/20181005-rib-TEM4/processing/bodymaker/bin2_of_bin6mb_B/SegOfLRM'
bin_dir   = 'automated_bodies'
# files
keyword='bodies_bin2_of_bin6mb_B_'
mrc2segpy    = bin_dir+'/mrc2seg.py'
input_mrc    = input_dir+'/relion_locres_filtered.mrc'
output_seg   = body_dir+'/'+keyword+'seg.mrc'
output_mask  = body_dir+'/'+keyword
# parameters
mrc_sdLevel     = 0.5
segger_nsteps   = 4
segger_stepsize = 5
segger_minregionsize = 100
segger_mincontactsize = 0
mask_blur   = 2.0

In [6]:
! /Applications/Chimera.app/Contents/MacOS/chimera --script "$mrc2segpy $input_mrc $output_seg $mrc_sdLevel $segger_nsteps $segger_stepsize $segger_minregionsize $segger_mincontactsize"

In [9]:
# break down: one mask per segment
seg2mask(output_seg, output_mask, sigma_blur = mask_blur)

In [ ]:
# manually decide which domain to keep
keep_mask   = np.array([True,False,False,True,False,True])

In [ ]:
# copy the ones we keep, and delete old
nkeep = seg2kept(output_seg, output_mask, keep_mask)

In [ ]:
# build mask of core domain
mask = data2mask(data_dry, sigma_blur = mask_blur)
data2mrc(output_mask+'kept_0.mrc',mask,mrc_template=input_mrc)

In [ ]:
# now apply a gaussian filter to soften the edges
for ikeep in np.arange(nkeep+1):
    mask = mrc2data(output_mask+'kept_'+str(ikeep)+'.mrc')
    mask_soft = data2mask(mask, sigma_blur = mask_blur) #gaussian_filter(mask, mask_blur)
    data2mrc(output_mask+str(ikeep)+'.mrc',mask_soft,mrc_template=output_mask+'kept_'+str(ikeep)+'.mrc')
    os.remove(output_mask+'kept_'+str(ikeep)+'.mrc')

# Toolkit

In [2]:
# routines on manipulation of mrc data

def mrc_stats(mrc_filename, get='std'):
    """ mrcstats
    """
    data = mrc2data(mrc_filename)
    mean = np.mean(data)
    if(get=='mean'):
        value = mean
    elif(get=='std'):
        value = np.std(data-mean)
    elif(get=='min'):
        value = np.min(data)
    elif(get=='max'):
        value = np.max(data)
    return value

def mrc_algebra(mrc1,mrc2,mrc_out,operation='add'):
    """mrc_algebra: mrc_out = mrc1 operation mrc2
    """
    data1 = mrc2data(mrc1)
    data2 = mrc2data(mrc2)
    if(operation=='add'):
        data = data1 + data2
    elif(operation=='subtract'):
        data = data1 - data2
    data2mrc(mrc_out,data,mrc_template=mrc1)
        
def mrc_select(mrc_filename, mode='above_value', value=0.):
    """mrc_select
    """
    data = mrc2data(mrc_filename)
    if(mode=='above_value'):
        data_selected = np.where(data >  value, data, 0.0)
    elif(mode=='equal_value'):
        data_selected = np.where(data == value, data, 0.0)
    else:
        data_selected = np.where(data <  value, data, 0.0)
    return data_selected
        
# routines mrc2bla or bla2mrc
    
def mrc2mask(mrc_filename, mask_filename, sigma_blur=0., threshold=0.1):
    """ mrc2mask: set to 1 any non-zero value, blurs, and binarize around threshold
    """
    data = mrc2data(mrc_filename)
    mask = data2mask(data, sigma_blur=sigma_blur, threshold=threshold)
    data2mrc(mask_filename, mask, mrc_template=mrc_filename)

def seg2mask(input_seg, output_key, sigma_blur = 0., threshold=0.1):
    """seg2mask
    """
    segments = mrc2data(input_seg)
    domains  = np.unique(segments).astype(int)
    if (domains.shape[0] > 1):
        for i in domains:
            if(i>0):
                data_domain = mrc_select(input_seg, mode='equal_value', value=i)
                mask = data2mask(data_domain, sigma_blur=sigma_blur, threshold=threshold)
                data2mrc(output_key+str(i)+'.mrc', mask, mrc_template=input_seg)

def seg2kept(input_seg, output_key, keep_mask):
    """seg2kep
    """
    ikeep=0
    segments = mrc2data(input_seg)
    domains  = np.unique(segments).astype(int)
    if (domains.shape[0] > 1):
        for i in domains:
            if(i>0):
                if keep_mask[i-1]:
                    ikeep += 1
                    shutil.copy(output_key+str(i)+'.mrc',output_key+'kept_'+str(ikeep)+'.mrc')
                os.remove(output_key+str(i)+'.mrc')
    print("keeping ",ikeep," domains")
    return ikeep
                
def data2mask(data, sigma_blur=0., threshold=0.1):
    """data2mask
    """
    mask = np.where(data > 0, 1.0, 0.0)
    if(sigma_blur > 0):
        mask = gaussian_filter(mask, sigma_blur)
        mask = np.where(mask > threshold, 1.0 , 0.0 ) #.astype(np.int8)
    return mask.astype(np.int8)
    
def mrc2data(mrc_filename):
    """ mrc2data
    """
    mrc  = mrcfile.open(mrc_filename, mode='r+')
    data = mrc.data
    mrc.close()
    return data

def data2mrc(mrc_filename,data,mrc_template=None):
    """ data2mrc
    """
    if mrc_template is None:
        print("Please provide a .mrc template to update data from")
    else:
        shutil.copy(mrc_template, mrc_filename)
        mrc = mrcfile.open(mrc_filename, mode='r+')
        mrc.data[:] = data
        mrc.close()